In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import logging
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from vegetation_indices import VegetationIndexCalculator
from risk_assessment import BushfireRiskAssessor
from visualization import BushfireRiskVisualizer
from database import BushfireRiskDatabase, test_database_connection
from main import BushfireRiskAssessment

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

print("✓ All libraries imported successfully")
print("✓ NSW Bushfire Risk Assessment modules loaded")


In [ ]:
# Initialize the assessment system
assessment = BushfireRiskAssessment(output_dir="./notebook_output")

# Load sample Sentinel-2 data
print("Loading synthetic Sentinel-2 data...")
bands = assessment.load_sample_data()

# Display information about the loaded data
print(f"✓ Loaded {len(bands)} spectral bands")
print(f"✓ Data shape: {bands['B04'].shape}")
print(f"✓ Bands available: {list(bands.keys())}")

# Show data statistics
print("\nBand Statistics:")
for band_name, band_data in bands.items():
    print(f"  {band_name}: Min={band_data.min():.0f}, Max={band_data.max():.0f}, Mean={band_data.mean():.0f}")


In [ ]:
# Calculate vegetation indices
print("Calculating vegetation indices...")
indices = assessment.calculate_vegetation_indices(bands)

# Display index statistics
print(f"✓ Calculated {len(indices)} vegetation indices")
print("\nVegetation Index Statistics:")
for index_name, index_data in indices.items():
    print(f"  {index_name}: Range {index_data.min():.3f} to {index_data.max():.3f}, Mean {index_data.mean():.3f}")

# Visualize the indices
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Vegetation Indices - Blue Mountains Study Area', fontsize=14, fontweight='bold')

index_configs = {
    'NDVI': {'cmap': 'RdYlGn', 'title': 'NDVI\n(Vegetation Health)'},
    'NDMI': {'cmap': 'Blues', 'title': 'NDMI\n(Moisture Content)'},
    'NBR': {'cmap': 'RdBu', 'title': 'NBR\n(Burn Ratio)'}
}

for i, (index_name, index_data) in enumerate(indices.items()):
    config = index_configs[index_name]
    im = axes[i].imshow(index_data, cmap=config['cmap'], vmin=-1, vmax=1)
    axes[i].set_title(config['title'])
    axes[i].axis('off')
    plt.colorbar(im, ax=axes[i], shrink=0.8)

plt.tight_layout()
plt.show()


In [ ]:
# Assess bushfire risk
print("Assessing bushfire risk...")
risk_scores, risk_categories = assessment.assess_risk(indices)

print(f"✓ Risk assessment completed")
print(f"✓ Risk scores range: {risk_scores.min():.1f} to {risk_scores.max():.1f}")
print(f"✓ Mean risk score: {risk_scores.mean():.1f}")

# Calculate area statistics
area_stats = assessment.risk_assessor.calculate_area_statistics(risk_categories)
print("\nRisk Category Distribution:")
for category, stats in area_stats.items():
    print(f"  {category}: {stats['percentage']:.1f}% ({stats['area_hectares']:.0f} ha)")

# Visualize risk assessment results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Risk scores (continuous)
im1 = ax1.imshow(risk_scores, cmap='YlOrRd', vmin=0, vmax=100)
ax1.set_title('Bushfire Risk Scores (0-100)', fontweight='bold')
ax1.axis('off')
cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.8)
cbar1.set_label('Risk Score')

# Risk categories (discrete)
risk_colors = ['#2E8B57', '#FFD700', '#FF8C00', '#FF4500', '#DC143C']
from matplotlib.colors import ListedColormap
cmap_cat = ListedColormap(risk_colors)

im2 = ax2.imshow(risk_categories, cmap=cmap_cat, vmin=1, vmax=5)
ax2.set_title('Risk Categories', fontweight='bold')
ax2.axis('off')
cbar2 = plt.colorbar(im2, ax=ax2, shrink=0.8, ticks=range(1, 6))
cbar2.set_label('Risk Category')
cbar2.set_ticklabels(['Very Low', 'Low', 'Moderate', 'High', 'Very High'])

plt.tight_layout()
plt.show()


In [ ]:
# Create interactive map
print("Creating interactive Folium map...")

# Define study area bounds
bounds = (assessment.study_area['min_lon'], assessment.study_area['min_lat'], 
         assessment.study_area['max_lon'], assessment.study_area['max_lat'])

# Create the interactive map
folium_map = assessment.visualizer.create_folium_map(
    risk_scores, 
    risk_categories, 
    bounds,
    center_lat=-33.75,
    center_lon=150.4,
    zoom_start=10
)

# Save the map to notebook output directory
map_path = Path("./notebook_output/interactive_map.html")
map_path.parent.mkdir(parents=True, exist_ok=True)
assessment.visualizer.save_folium_map(folium_map, map_path)

print(f"✓ Interactive map created and saved to: {map_path}")
print("✓ Open the HTML file in a web browser to explore the interactive map")

# Display the map (if running in a Jupyter environment that supports it)
try:
    folium_map
except:
    print("Note: Map display may not work in all environments. Check the saved HTML file.")


In [ ]:
# Test database connection and save results (optional)
print("Testing database connection...")

if test_database_connection():
    print("✓ Database connection successful")
    
    # Save results to database
    db_record_id = assessment.save_to_database(risk_scores, risk_categories, indices)
    
    if db_record_id:
        print(f"✓ Assessment saved to database with ID: {db_record_id}")
        
        # Query some statistics
        from database import setup_database_from_env
        with setup_database_from_env() as db:
            stats = db.get_risk_statistics()
            print(f"✓ Total assessments in database: {stats['total_assessments']}")
            print(f"✓ Average risk score: {stats['avg_risk_score']:.1f}")
    else:
        print("⚠ Failed to save to database")
        
else:
    print("⚠ Database connection failed - skipping database operations")
    print("  To enable database features:")
    print("  1. Install PostgreSQL with PostGIS extension")
    print("  2. Create a database named 'bushfire_risk'")
    print("  3. Set environment variables: DB_HOST, DB_NAME, DB_USER, DB_PASSWORD")


In [ ]:
# Final summary
print("=" * 60)
print("NSW BUSHFIRE RISK ASSESSMENT - COMPLETED")
print("=" * 60)
print(f"Study Area: Blue Mountains, NSW")
print(f"Data Shape: {risk_scores.shape}")
print(f"Mean Risk Score: {risk_scores.mean():.1f}/100")
print(f"High Risk Areas: {np.sum(risk_categories >= 4) / risk_categories.size * 100:.1f}%")
print(f"Output Directory: ./notebook_output/")
print("=" * 60)

# List all generated files
output_dir = Path("./notebook_output")
if output_dir.exists():
    files = list(output_dir.glob("*"))
    if files:
        print(f"Generated {len(files)} output files:")
        for file in files:
            print(f"  📁 {file.name}")
    else:
        print("Output directory created but no files generated yet.")

print("\n🎯 Demonstration completed successfully!")
print("🚀 Ready for production deployment with real Sentinel-2 data")
